In [2]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from problem_generator import Problem_Generate_Model
from langchain.schema import HumanMessage, SystemMessage
from util import random_word
from util import find_random_question
import re

In [3]:
total_cost = 0
default_model = "gpt-4o-mini"
verbose = True
models = [default_model]
problem_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=0.1, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)

main_title = random_word()
if verbose:
    print(main_title)

problem_generate_model.request_models_responses(
    [
        SystemMessage(content="주제어와 관련된 일상적인 단어 1개를 제시하라."),
        HumanMessage(content="주제어: " + main_title + " 단어: "),
    ]
)
word_responses = problem_generate_model.get_model_responses()
word = word_responses[0][1]
def clean_text(text):
            pattern = r':\s*(.*)$'
            match = re.search(pattern, text)
            if match:
                return match.group(1).strip()
            text = re.sub(r'\n', ' ', text)  # 개행 문자를 공백으로 대체
            return text.strip()
word = clean_text(word)
total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', word_responses[0][2]).group(1))

keyword = 'TOPIK_2_READING'
problem_type = 'READING_2_PROBLEM_TYPE_4'
random_question = find_random_question(keyword, problem_type, detail_type=1, count=2, api_key_path="./../API_KEY/topik_api_key.json")
if verbose:
    print(random_question)
    
example_dict = []
for item in random_question:
    example_text = item['example']
    problem_generate_model.request_models_responses([
    SystemMessage(content="문장의 주제 단어 1개를 제시하라."),
    HumanMessage(content="문장: "+ example_text + "주제 단어: "),
        ]
    )
    total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', problem_generate_model.get_model_responses()[0][2]).group(1))
    title_responses = problem_generate_model.get_model_responses()[0][1]
    temp = {"주제": title_responses, "예문": example_text, "선택지": ', '.join(item['selector']), "정답": item['answer']}
    example_dict.append(temp)
if verbose:
    print(example_dict)
example_prompt = ""
for item in example_dict:
    example_str = f"주제: {item['주제']}, 예문: {item['예문']}\n 선택지: {item['선택지']}\n 정답: {item['정답']}"
    example_prompt += example_str + "\n"
if verbose:
    print(example_prompt)
problem_generate_model.request_models_responses(
    [
        SystemMessage(
            content="#너는 문제를 출제하는 선생님이야. 예시처럼 주어지는 주제에 대한 글의 올바른 내용 고르기 문제 1개를 생성해줘. #답변 예시 "
            + example_prompt
        ),
        HumanMessage(content="주제: " + word),
    ]
)
problem_responses = problem_generate_model.get_model_responses()
total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', problem_responses[0][2]).group(1))
if verbose:
    print(problem_responses[0][1])
    
def extract_parts(text):
    # 예문 추출
    example_pattern = r'예문:\s*(.*)'
    example_match = re.search(example_pattern, text)
    example = example_match.group(1).strip() if example_match else None

    # 선택지 추출
    options_pattern = r'선택지:\s*(.*)'
    options_match = re.search(options_pattern, text)
    if options_match:
        options = options_match.group(1).strip().split(', ')
        for option in options:
            option = clean_text(option)
    else:
        options = []

    # 정답 추출
    answer_pattern = r'정답:\s*(.*)'
    answer_match = re.search(answer_pattern, text)
    answer = answer_match.group(1).strip() if answer_match else None

    return example, options, answer

example, options, answer = extract_parts(problem_responses[0][1])

result = {
    "example": example,
    "selector": options,
    "answer": answer,
    "eval_answer": answer,
    "eval_explain": example,
    "total_cost": total_cost
}
print(result)

신세
gpt-4o-mini | 
부담

OpenAI Tokens Used: 43
    Prompt Tokens: 41
    Completion Tokens: 2
Successful Requests: 1
Total Cost (USD): $3e-05
Total Cost (KRW): ₩0.0405
--------------------------------------------------------------------------------------------------
[{'example': '종이 신문을 읽는 가구가 계속 감소하고 있는 것으로 나타났다. 종이 신문을 읽는 집은 지난해보다 5% 정도 감소해서 다섯 집 중 한 집만이 배달해서 읽고 있었다. 인터넷으로 언제든지 뉴스를 볼 수 있게 되면서 종이 신문을 보는 사람이 감소하고 있는 것이다. 한편 이번 조사는 지난해에 조사에 참여한 집으로 전화를 거는 방식으로 진행되었다.', 'selector': ['이번 조사는 직접 방문해서 조사하는 방식을 사용했다.', '종이 신문을 읽는 사람이 줄면서 인터넷의 사용도 줄었다.', '조사된 가구 중에서 절반은 집으로 배달되는 신문을 읽는다.', '이번 조사와 지난해 조사는 서로 같은 가구를 대상으로 했다.'], 'answer': '이번 조사와 지난해 조사는 서로 같은 가구를 대상으로 했다.'}, {'example': '주말마다 서울 광화문 주변이 콘서트장으로 바뀐다. 바로 ‘광화문 콘서 트’라는 프로그램 때문이다. 이 프로그램은 매회 가수 한 명이 출연해 10여 곡의 노래를 부르는 음악 프로그램으로 다음 주 첫 방송을 앞두고 있다. 녹화는 광화문 앞 무대에서 일요일 오후 7시부터 시작되며 방청 신청은 프로그램 홈페이지를 통해 300명까지 선착순으로 받는다.', 'selector': ['이 프로그램은 평일에 녹화를 진행한다.', '광화문 콘서트는 오래된 음악 프로그램이다.', '직접 가서 보려면 인터넷으로 신청해야 한다.', '이 프로그램에는 여러 명의 가수가 출연한다.

In [2]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from problem_generator import Problem_Generate_Model
from langchain.schema import HumanMessage, SystemMessage
from util import random_word
from util import find_random_question
import re

In [5]:
total_cost = 0
verbose = True
default_model = "gpt-4o-mini"

keyword = 'TOPIK_2_READING'
problem_type = 'READING_2_PROBLEM_TYPE_4'
random_question = find_random_question(keyword, problem_type, detail_type=2, count=1, api_key_path="./../API_KEY/topik_api_key.json")
if verbose:
    print(random_question)
random_question = random_question[0]
random_question_selector = random_question['selector']
if verbose:
    print(random_question_selector)

models = [default_model]
problem_generate_model = Problem_Generate_Model(
    models, use_cache=True, temperature=1, verbose=verbose, api_key_path="./../API_KEY/llm_api_key.json"
)
selector = []
for select in random_question_selector:
    if verbose:
        print(f"selector: {select}")
    problem_generate_model.request_models_responses(
        [
            SystemMessage(content="주어지는 문장을 동일한 뜻을 지닌 다른 문장으로 바꿔줘. 대치법, 동의어, 유의어 등을 활용하면 좋아."),
            HumanMessage(content="바꿀 문장: " + select + " 바뀐 문장: "),
        ]
    )
    word_responses = problem_generate_model.get_model_responses()
    word = word_responses[0][1]
    def clean_text(text):
        pattern = r':\s*(.*)$'
        match = re.search(pattern, text)
        if match:
            return match.group(1).strip()
        text = re.sub(r'\n', ' ', text)  # 개행 문자를 공백으로 대체
        return text.strip()
    word = clean_text(word)
    total_cost += float(re.search(r'Total Cost \(KRW\): ₩([\d.]+)', word_responses[0][2]).group(1))
    selector.append(word)
    
    if select == random_question['answer']:
        random_question['answer'] = word
    
if verbose:
    print(selector)

result = {
    "example": random_question['example'],
    "selector": selector,
    "answer": random_question['answer'],
    "eval_answer": random_question['answer'],
    "eval_explain": "",
    "total_cost": total_cost,
}

print(result)

[{'example': 'https://topikkorea.s3.amazonaws.com/topik_image/5c23f762-23ef-11ef-81fc-e5a43cd400fd.png', 'selector': ['신문을 보는 사람의 비율은 20대와 60대가 같다.', '모든 세대가 텔레비전보다 신문을 더 많이 본다.', '신문을 보는 사람의 비율은 60대가 40대보다 낮다.', '텔레비전을 보는 사람의 비율은 20대가 40대보다 높다.'], 'answer': '신문을 보는 사람의 비율은 60대가 40대보다 낮다.'}]
['신문을 보는 사람의 비율은 20대와 60대가 같다.', '모든 세대가 텔레비전보다 신문을 더 많이 본다.', '신문을 보는 사람의 비율은 60대가 40대보다 낮다.', '텔레비전을 보는 사람의 비율은 20대가 40대보다 높다.']
selector: 신문을 보는 사람의 비율은 20대와 60대가 같다.
gpt-4o-mini | 
신문을 읽는 사람의 비율은 20대와 60대가 동일하다.

OpenAI Tokens Used: 100
    Prompt Tokens: 80
    Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $8e-05
Total Cost (KRW): ₩0.108
--------------------------------------------------------------------------------------------------
selector: 모든 세대가 텔레비전보다 신문을 더 많이 본다.
gpt-4o-mini | 
모든 연령층이 신문을 텔레비전보다 더 자주 읽는다.

OpenAI Tokens Used: 100
    Prompt Tokens: 79
    Completion Tokens: 21
Successful Requests: 1
Total Cost (USD): $8e-05
Total Cost (KRW): ₩0.108
-------------------